# Model

## Load data

In [2]:
import pandas as pd 
import numpy as np 

df=pd.read_csv(fr'..\data\cleaned_data_08_2025.csv')

display(df.head())

,index,propertyCode,price,log_price,nom_comar,municipality,propertyType,size,cat_size,rooms,bathrooms,floor_grouped,hasLift,latitude,longitude
0,0,108933600,300000.0,12.611541,Baix Llobregat,Esplugues de Llobregat,flat,78.0,Mediana,4,1,3-4,1,41.370454,2.083978
1,1,108932312,650000.0,13.384729,Vallès Occidental,Santa Perpètua de Mogoda,chalet,243.0,Muy Grande,4,3,0,0,41.525747,2.192719
2,2,108932560,570000.0,13.253393,Maresme,Pineda de Mar,duplex,208.0,Muy Grande,4,3,5+,1,41.617925,2.673653
3,3,108933084,390600.0,12.875442,Maresme,Cabrils,chalet,166.0,Muy Grande,3,2,0,0,41.514466,2.381917
4,4,108933074,250000.0,12.429220,Anoia,Masquefa,chalet,212.0,Muy Grande,4,2,0,0,41.489203,1.807568


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4824 entries, 0 to 4823
Data columns (total 15 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   index          4824 non-null   int64  
 1   propertyCode   4824 non-null   int64  
 2   price          4824 non-null   float64
 3   log_price      4824 non-null   float64
 4   nom_comar      4824 non-null   object 
 5   municipality   4824 non-null   object 
 6   propertyType   4824 non-null   object 
 7   size           4824 non-null   float64
 8   cat_size       4824 non-null   object 
 9   rooms          4824 non-null   int64  
 10  bathrooms      4824 non-null   int64  
 11  floor_grouped  4824 non-null   object 
 12  hasLift        4824 non-null   int64  
 13  latitude       4824 non-null   float64
 14  longitude      4824 non-null   float64
dtypes: float64(5), int64(5), object(5)
memory usage: 565.4+ KB


In [4]:
df.describe()

,index,propertyCode,price,log_price,size,rooms,bathrooms,hasLift,latitude,longitude
count,4824.000000,4.824000e+03,4824.000000,4824.000000,4824.000000,4824.000000,4824.000000,4824.000000,4824.000000,4824.000000
mean,2498.090796,1.084854e+08,327753.444237,12.567028,110.377902,2.998342,1.610282,0.487769,41.462998,2.127987
std,1442.780686,2.353131e+06,172884.186205,0.518278,71.988356,1.129036,0.770777,0.499902,0.133644,0.201374
min,0.000000,3.099994e+07,90000.000000,11.407576,20.000000,0.000000,0.000000,0.000000,41.196342,1.511590
25%,1254.750000,1.087952e+08,194355.500000,12.177449,68.000000,2.000000,1.000000,0.000000,41.376164,2.038863
50%,2490.500000,1.088441e+08,280000.000000,12.542548,88.000000,3.000000,1.000000,0.000000,41.432072,2.144660
75%,3751.250000,1.088904e+08,420000.000000,12.948012,120.000000,4.000000,2.000000,1.000000,41.546522,2.201214
max,4999.000000,1.089336e+08,850000.000000,13.652993,759.000000,10.000000,10.000000,1.000000,42.256445,2.764981


In [ ]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import RidgeCV
from sklearn.metrics import mean_squared_error, r2_score


# =========================================================
# 0) Separación de variables (excluimos IDs de las features)
# =========================================================
id_cols = ['propertyCode', 'index']         # las guardamos para el reporte
target_cols = ['price', 'log_price']        # target en log
drop_from_X = id_cols + target_cols

# X e y
X = df.drop(columns=drop_from_X, errors='ignore').copy()
y = df['log_price'].copy()

# =========================================================
# 1) Holdout estratificado por nom_comar
# =========================================================
# (aseguramos que exista la columna para estratificar)
assert 'nom_comar' in df.columns, "'nom_comar' no está en el DataFrame."

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2,
    stratify=df['nom_comar'],
    random_state=42
)

# Guardamos IDs y contexto para el reporte
test_ids = df.loc[X_test.index, id_cols]          # propertyCode + index
test_context = df.loc[X_test.index, ['nom_comar','municipality']]

# =========================================================
# 2) Preprocesamiento: OHE SOLO para 'nom_comar', resto numéricas
# =========================================================
nom_comar_col = ['nom_comar']
num_features = X.select_dtypes(include=['int64','float64']).columns.tolist()

preprocessor = ColumnTransformer(
    transformers=[
        ('ohe_nom', OneHotEncoder(drop='first', handle_unknown='ignore'), nom_comar_col),
        ('num', 'passthrough', num_features),
    ],
    remainder='drop'  # descarta otras object (municipality, propertyType, etc.)
)

# =========================================================
# 3) Modelo: Ridge con CV interna (robusto a multicolinealidad)
# =========================================================
alphas = np.logspace(-3, 3, 13)  # 0.001 a 1000
model = Pipeline(steps=[
    ('prep', preprocessor),
    ('ridge', RidgeCV(alphas=alphas, cv=5, scoring='neg_root_mean_squared_error'))
])

# Entrenamos
model.fit(X_train, y_train)

# =========================================================
# 4) Predicción y métricas en log y en precio
# =========================================================
y_pred_log = model.predict(X_test)
y_pred_price = np.exp(y_pred_log)
y_test_price = np.exp(y_test)

r2_log = r2_score(y_test, y_pred_log)
rmse_log = np.sqrt(mean_squared_error(y_test, y_pred_log))
rmse_price = np.sqrt(mean_squared_error(y_test_price, y_pred_price))

def rmse(y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true, y_pred))

r2_log = r2_score(y_test, y_pred_log)
print(f"R² (log): {r2_log:.4f}")
print(f"RMSE (log): {rmse(y_test, y_pred_log):.4f}")
print(f"RMSE (price): {rmse(y_test_price, y_pred_price):,.2f}")
print("Alpha elegido (Ridge):", model.named_steps['ridge'].alpha_)

# =========================================================
# 5) DataFrame de resultados con propertyCode para buscar en Idealista
# =========================================================
preds = pd.DataFrame({
    'y_true_log': y_test.values,
    'y_pred_log': y_pred_log,
    'price_true': y_test_price.values,
    'price_pred': y_pred_price
}, index=X_test.index)

# Añadimos IDs y contexto útil
preds = pd.concat([test_ids, test_context, preds], axis=1)

# Orden sugerido
cols_order = ['propertyCode', 'nom_comar', 'municipality',
                'price_true', 'price_pred', 'y_true_log', 'y_pred_log', 'index']
preds = preds[cols_order]

# Ejemplo: ver las 10 discrepancias más altas para inspeccionar
preds['abs_error'] = (preds['price_true'] - preds['price_pred']).abs()
preds = preds.sort_values('abs_error', ascending=False)

preds.head(10)


R² (log): 0.5840
RMSE (log): 0.3322
RMSE (price): 131,715.91
Alpha elegido (Ridge): 0.01


,propertyCode,nom_comar,municipality,price_true,price_pred,y_true_log,y_pred_log,index,abs_error
3468,108823055,Vallès Occidental,Sabadell,768001.0,1.735449e+06,13.551546,14.366777,3602,967448.366118
219,108925117,Barcelonès,Barcelona,400001.0,1.273041e+06,12.899222,14.056919,226,873040.396079
2765,108846397,Maresme,Sant Pol de Mar,679001.0,1.528961e+06,13.428378,14.240099,2863,849960.484778
1466,108889988,Garraf,Canyelles,755001.0,1.532561e+06,13.534474,14.242451,1519,777559.787399
2897,108842914,Vallès Occidental,Sabadell,840001.0,1.594580e+06,13.641158,14.282121,2998,754579.095034
1607,108885346,Vallès Oriental,L'Ametlla del Vallès,795001.0,1.341299e+06,13.586099,14.109149,1663,546297.585672
2150,108863280,Maresme,Sant Vicenç de Montalt,850001.0,3.536476e+05,13.652993,12.776056,2221,496353.430962
3165,108833639,Barcelonès,Barcelona,789001.0,3.024502e+05,13.578523,12.619672,3282,486550.820577
3694,108816505,Barcelonès,Barcelona,789001.0,3.025335e+05,13.578523,12.619947,3829,486467.501318
4614,108784707,Baix Llobregat,Viladecans,795001.0,3.286052e+05,13.586099,12.702612,4777,466395.812322


In [5]:
import numpy as np
import pandas as pd

from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import RidgeCV
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import r2_score, mean_squared_error

# =========================
# 0) COPIA + LIMPIEZA BÁSICA
# =========================
df2 = df.copy()

# --- A) Encoding ORDINAl para floor_grouped y cat_size ---
floor_map = {'0':0, 0:0, '1':1, 1:1, '2':2, 2:2, '3-4':3, '5+':4}
size_map  = {'Pequeña':0, 'Mediana':1, 'Grande':2, 'Muy Grande':3}

df2['floor_num'] = df2['floor_grouped'].map(floor_map).fillna(0).astype(int)
df2['size_cat']  = df2['cat_size'].map(size_map).fillna(1).astype(int)  # por defecto 'Mediana'

# --- B) Features de interacción y ratios ---
df2['lift_x_floor']   = df2['hasLift'] * df2['floor_num']
df2['rooms_per_100m'] = df2['rooms'] / (df2['size'] / 100.0)
df2['bath_per_room']  = np.where(df2['rooms'] > 0, df2['bathrooms'] / df2['rooms'], 0.0)

# Reemplazos de infinitos/NaN por 0:
for c in ['rooms_per_100m','bath_per_room']:
    df2[c] = df2[c].replace([np.inf, -np.inf], np.nan).fillna(0.0)

# --- C) Distancia geográfica (Haversine) a Plaça de Catalunya (Barcelona) ---
# Ref: 41.3870, 2.1700
R = 6371.0  # radio Tierra (km)
lat0 = np.deg2rad(41.3870); lon0 = np.deg2rad(2.1700)
lat  = np.deg2rad(df2['latitude'].values); lon = np.deg2rad(df2['longitude'].values)
dlat = lat - lat0; dlon = lon - lon0
a = np.sin(dlat/2)**2 + np.cos(lat0)*np.cos(lat)*np.sin(dlon/2)**2
c = 2*np.arctan2(np.sqrt(a), np.sqrt(1-a))
df2['dist_center_km'] = R * c

# --- D) Frequency encoding para municipality ---
muni_freq = df2['municipality'].value_counts(normalize=True)
df2['municipality_freq'] = df2['municipality'].map(muni_freq).fillna(0.0)

# --- E) Winsorización ligera de outliers en numéricas clave ---
def winsorize(s, p_low=0.01, p_high=0.99):
    lo, hi = s.quantile(p_low), s.quantile(p_high)
    return s.clip(lo, hi)

for col in ['size','rooms','bathrooms','dist_center_km','rooms_per_100m','bath_per_room']:
    df2[col] = winsorize(df2[col])

# =========================
# 1) DEFINIR X, y, IDs
# =========================
id_cols = ['propertyCode', 'index']
target  = 'log_price'

X = df2.drop(columns=['price', 'log_price', 'propertyCode', 'index',
                      # quitamos las categóricas originales que ya codificamos
                      'floor_grouped','cat_size','municipality','propertyType'])
y = df2[target].copy()
strata = df2['nom_comar']  # para CV estratificada por comarca

# =========================
# 2) PREPROCESAMIENTO (OHE SOLO nom_comar)
# =========================
ohe_cols = ['nom_comar']
num_cols = X.columns.drop(ohe_cols).tolist()

preprocessor = ColumnTransformer(
    transformers=[
        ('ohe_nom', OneHotEncoder(drop='first', handle_unknown='ignore'), ohe_cols),
        ('num', 'passthrough', num_cols),
    ],
    remainder='drop'
)

# =========================
# 3) MODELOS
# =========================
alphas = np.logspace(-3, 3, 13)
ridge = Pipeline(steps=[
    ('prep', preprocessor),
    ('model', RidgeCV(alphas=alphas, cv=5, scoring='neg_mean_squared_error'))
])

gbr = Pipeline(steps=[
    ('prep', preprocessor),
    ('model', GradientBoostingRegressor(
        n_estimators=500, max_depth=3, learning_rate=0.05, subsample=0.8,
        random_state=42
    ))
])

models = {'RidgeCV': ridge, 'GBR': gbr}

# =========================
# 4) CV estratificada por nom_comar
# =========================
def rmse(y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true, y_pred))

def cv_eval(model, X, y, strata, n_splits=5, random_state=42):
    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=random_state)
    r2_logs, rmse_logs, rmse_prices = [], [], []

    for train_idx, test_idx in skf.split(X, strata):
        X_tr, X_te = X.iloc[train_idx], X.iloc[test_idx]
        y_tr, y_te = y.iloc[train_idx], y.iloc[test_idx]

        model.fit(X_tr, y_tr)
        y_pred_log = model.predict(X_te)

        # métricas en log
        r2_logs.append(r2_score(y_te, y_pred_log))
        rmse_logs.append(rmse(y_te, y_pred_log))

        # métricas en € (volver de log)
        y_te_price   = np.exp(y_te)
        y_pred_price = np.exp(y_pred_log)
        rmse_prices.append(rmse(y_te_price, y_pred_price))

    return {
        'R2_log_mean': np.mean(r2_logs), 'R2_log_std': np.std(r2_logs),
        'RMSE_log_mean': np.mean(rmse_logs), 'RMSE_log_std': np.std(rmse_logs),
        'RMSE_price_mean': np.mean(rmse_prices), 'RMSE_price_std': np.std(rmse_prices),
    }

results = {}
for name, m in models.items():
    results[name] = cv_eval(m, X, y, strata)
    print(f"\n=== {name} (CV estratificada por nom_comar) ===")
    print(f"R² (log):   {results[name]['R2_log_mean']:.4f} ± {results[name]['R2_log_std']:.4f}")
    print(f"RMSE (log): {results[name]['RMSE_log_mean']:.4f} ± {results[name]['RMSE_log_std']:.4f}")
    print(f"RMSE (€):   {results[name]['RMSE_price_mean']:,.2f} ± {results[name]['RMSE_price_std']:,.2f}")

# =========================
# 5) ENTRENAR MEJOR MODELO + REPORTE CON propertyCode
# =========================
# Elige el mejor por R2 log (puedes cambiar criterio)
best_name = max(results, key=lambda k: results[k]['R2_log_mean'])
best_model = models[best_name].fit(X, y)

print(f"\nMejor modelo por CV: {best_name}")

# Predicciones “out-of-sample” via CV (para ranking de errores)
# Si prefieres, separa un holdout aparte.
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
oof_pred = pd.Series(index=X.index, dtype=float)

for train_idx, test_idx in skf.split(X, strata):
    best_model_fold = models[best_name]
    best_model_fold.fit(X.iloc[train_idx], y.iloc[train_idx])
    oof_pred.iloc[test_idx] = best_model_fold.predict(X.iloc[test_idx])

# Reporte con IDs y contexto
report = pd.DataFrame({
    'propertyCode': df2['propertyCode'],
    'nom_comar': df2['nom_comar'],
    'municipality': df2['municipality'],
    'price_true': np.exp(y),
    'price_pred': np.exp(oof_pred),
})
report['abs_error'] = (report['price_true'] - report['price_pred']).abs()
report = report.sort_values('abs_error', ascending=False)

# Top 10 propiedades con mayor error (para auditar)
report.head(10)



=== RidgeCV (CV estratificada por nom_comar) ===
R² (log):   0.6942 ± 0.0181
RMSE (log): 0.2862 ± 0.0048
RMSE (€):   100,876.51 ± 3,216.99

=== GBR (CV estratificada por nom_comar) ===
R² (log):   0.7919 ± 0.0135
RMSE (log): 0.2360 ± 0.0052
RMSE (€):   80,839.00 ± 2,721.60

Mejor modelo por CV: GBR


,propertyCode,nom_comar,municipality,price_true,price_pred,abs_error
23,108931814,Maresme,Montgat,350001.0,873559.722453,523558.722453
2217,108861873,Barcelonès,Barcelona,790001.0,311928.636931,478072.363069
1117,108899513,Maresme,Mataró,850001.0,376368.207926,473632.792074
2150,108863280,Maresme,Sant Vicenç de Montalt,850001.0,376404.441901,473596.558099
719,108556699,Garraf,Cubelles,850001.0,422704.349565,427296.650435
1491,102207079,Garraf,Sitges,359376.0,769193.066905,409817.066905
4442,108790284,Alt Penedès,Castellet i la Gornal,635001.0,226241.100241,408759.899759
4614,108784707,Baix Llobregat,Viladecans,795001.0,388839.174057,406161.825943
3304,108828399,Garraf,Sitges,750001.0,348711.257965,401289.742035
3867,108809040,Osona,Gurb,710001.0,313741.920478,396259.079522


In [ ]:
report[report['price_true']<report['price_pred']]

,propertyCode,nom_comar,municipality,price_true,price_pred,abs_error
23,108931814,Maresme,Montgat,350001.0,873559.722453,523558.722453
1491,102207079,Garraf,Sitges,359376.0,769193.066905,409817.066905
4522,108787717,Barcelonès,Barcelona,110001.0,458521.763458,348520.763458
4624,108784433,Barcelonès,Barcelona,270001.0,610542.529343,340541.529343
1056,108900954,Barcelonès,Barcelona,195001.0,508448.167829,313447.167829
...,...,...,...,...,...,...
2955,108840775,Maresme,Premià de Dalt,745001.0,745093.068905,92.068905
4421,108790704,Maresme,Canet de Mar,269681.0,269753.996001,72.996001
2268,105425541,Vallès Oriental,Mollet del Vallès,205001.0,205047.193322,46.193322
2953,108840904,Vallès Occidental,Rubí,189001.0,189036.562094,35.562094


In [10]:
# Definimos "subvaluada" como precio_real < precio_predicho * (1 - umbral)
umbral = 0.10  # 10% por debajo del valor de mercado estimado

subvaluadas = report[report['price_true'] < report['price_pred'] * (1 - umbral)].copy()
subvaluadas['diff_%'] = 100 * (subvaluadas['price_pred'] - subvaluadas['price_true']) / subvaluadas['price_pred']

# Ordenamos por mayor porcentaje de diferencia
subvaluadas = subvaluadas.sort_values('diff_%', ascending=False)

# Mostramos las top 15 oportunidades
cols_mostrar = ['propertyCode','nom_comar','municipality','price_true','price_pred','diff_%']
display(subvaluadas[cols_mostrar])

# Ruta de guardado
output_path = r"..\data"
# Opcional: exportar a Excel para consulta en Idealista
subvaluadas.to_excel(f"{output_path}\\propiedades_subvaluadas.xlsx", index=False)
subvaluadas.to_csv(f"{output_path}\\propiedades_subvaluadas.xlsx", index=False)


,propertyCode,nom_comar,municipality,price_true,price_pred,diff_%
4522,108787717,Barcelonès,Barcelona,110001.0,458521.763458,76.009645
1906,108873235,Baix Llobregat,Sant Joan Despí,118001.0,333234.072988,64.589155
1056,108900954,Barcelonès,Barcelona,195001.0,508448.167829,61.647811
23,108931814,Maresme,Montgat,350001.0,873559.722453,59.933936
1427,108891086,Barcelonès,Barcelona,95001.0,236756.005070,59.873879
...,...,...,...,...,...,...
1204,108896203,Maresme,Premià de Dalt,475001.0,528262.292721,10.082357
2760,108844998,Barcelonès,Barcelona,459001.0,510435.808080,10.076646
4418,108790632,Vallès Oriental,Cànoves i Samalús,495001.0,550314.257260,10.051213
1566,108231416,Barcelonès,Barcelona,458001.0,509151.995514,10.046312


In [14]:
import numpy as np
import pandas as pd

# -------------------------------
# 1) Enlace directo a Idealista
# -------------------------------
# Nota: ajusta el dominio si usas idealista.es u otro subdominio
def idealista_url(property_code, domain="https://www.idealista.com/inmueble"):
    return f"{domain}/{int(property_code)}/"

report = report.copy()
report['idealista_url'] = report['propertyCode'].apply(idealista_url)

# ---------------------------------------------------
# 2) Métricas de infravaloración y “sorpresa” (zscore)
# ---------------------------------------------------
# % de subvaloración respecto a la predicción (en €)
report['undervaluation_pct'] = 100.0 * (report['price_pred'] - report['price_true']) / report['price_pred']

# Residuo en log (pred - real) para medir sorpresa estadística
# (si solo tienes precios, calcula logs al vuelo)
report['y_pred_log'] = np.log(report['price_pred'])
report['y_true_log'] = np.log(report['price_true'])
report['res_log']    = report['y_pred_log'] - report['y_true_log']

# z-score del residuo (más alto => más “atípicamente” barata)
res_std = report['res_log'].std(ddof=1)
report['res_zscore'] = np.where(res_std > 0, report['res_log'] / res_std, 0.0)

# -------------------------------------------
# 3) Regla de negocio para “subvaluada”
# -------------------------------------------
UMBRAL_PCT = 10.0  # >=10% por debajo del valor estimado
subvaluadas = report.query('undervaluation_pct >= @UMBRAL_PCT').copy()

# Ranking: combinamos magnitud (% subval) y sorpresa (z-score)
# Peso 70/30 como punto de partida (ajústalo según tu apetito de riesgo)
subvaluadas['priority_score'] = 0.7*subvaluadas['undervaluation_pct'] + 0.3*(100*subvaluadas['res_zscore'])

# Orden final
cols_show = [
    'propertyCode','nom_comar','municipality','price_true','price_pred',
    'undervaluation_pct','res_zscore','priority_score','idealista_url'
]
subvaluadas = subvaluadas.sort_values(['priority_score','undervaluation_pct','res_zscore'], ascending=False)

# Vista rápida (Top 15)
display_cols = cols_show
print(subvaluadas[display_cols].head(15).to_string(index=False))

# -------------------------------------------
# 4) Top oportunidades por comarca (resumen)
# -------------------------------------------
topk_by_comarca = (
    subvaluadas
    .sort_values('priority_score', ascending=False)
    .groupby('nom_comar')
    .head(3)[display_cols]
)

print("\nTop 3 oportunidades por comarca:")
print(topk_by_comarca.to_string(index=False))

# -------------------------------------------
# 5) Exportaciones
# -------------------------------------------
# Ruta de guardado
output_path = r"..\data"

# Guarda todas las propiedades subvaluadas
subvaluadas.to_excel(f"{output_path}\\propiedades_subvaluadas.xlsx", index=False)
subvaluadas.to_csv(f"{output_path}\\propiedades_subvaluadas.csv", index=False)

# Guarda el top 3 por comarca
topk_by_comarca.to_excel(f"{output_path}\\top3_por_comarca.xlsx", index=False)
topk_by_comarca.to_csv(f"{output_path}\\top3_por_comarca.csv", index=False)

print(f"Archivos guardados en {output_path}")



 propertyCode       nom_comar            municipality  price_true    price_pred  undervaluation_pct  res_zscore  priority_score                                 idealista_url
    108787717      Barcelonès               Barcelona    110001.0 458521.763458           76.009645    6.046041      234.587971 https://www.idealista.com/inmueble/108787717/
    108873235  Baix Llobregat         Sant Joan Despí    118001.0 333234.072988           64.589155    4.396938      177.120552 https://www.idealista.com/inmueble/108873235/
    108900954      Barcelonès               Barcelona    195001.0 508448.167829           61.647811    4.058985      164.923011 https://www.idealista.com/inmueble/108900954/
    108931814         Maresme                 Montgat    350001.0 873559.722453           59.933936    3.873823      158.168453 https://www.idealista.com/inmueble/108931814/
    108891086      Barcelonès               Barcelona     95001.0 236756.005070           59.873879    3.867480      157.936101 ht